# API 4-2: REST APIs 

Non-GET, Caching Strategies, Azure 

## HTTP Methods beyond GET

So far we have only seen GET requests. But there are other HTTP methods that are used to perform different operations  such as:

- POST: Create a new resource
- PUT: Update an existing resource
- DELETE: Delete a resource

From the perspective of `requests` these behave the same as a GET. The only subtle difference is that you can pass a `data` parameter when there are large payloads which cannot fit on the URL.  


## Example of a POST request

Let's do an example with the Azure sentiment API from the IoT Portal. This REST API requires the text to analyze, and will return the sentiment or "mood" of the text.  Since there can be a substantial amount of text, the API requires the POST method, and the data to be sent in the body of the request.

First, Try it out in the IoT portal:

`/api/azure/sentiment`

Text: "I love IST356. It is the best course I've ever taken."

Then, let's convert the `curl` command to Python `requests` code. 

In [3]:
import requests

'''
curl -X 'POST' \
  'https://cent.ischool-iot.net/api/azure/sentiment' \
  -H 'accept: application/json' \
  -H 'X-API-KEY: APIKEY' \
  -H 'Content-Type: application/x-www-form-urlencoded' \
  -d 'text=I%20love%20IST356.%20It%20is%20the%20best%20course%20I'\''ve%20ever%20taken.'
'''

apikey = 'YOURAPIKEYHERE'
url = 'https://cent.ischool-iot.net/api/azure/sentiment'
headers = {
    'accept': 'application/json',
    'X-API-KEY': apikey,
    'Content-Type': 'application/x-www-form-urlencoded'
}
data = {
    'text': 'I love IST356. It is the best course I\'ve ever taken.'
}
response = requests.post(url, headers=headers, data=data)
response.raise_for_status()
results = response.json()
sentiment = results['results']['documents'][0]['sentiment']
print(sentiment)

HTTPError: 500 Server Error: INTERNAL SERVER ERROR for url: https://cent.ischool-iot.net/api/azure/sentiment

## Challenge 4-2-1

For this challenge, use Azure entity recognition API to extract entities from the following text.

"The Dallas Cowboys are a far better team than the New York Giants this year. The Giants have not won a conference game yet."

Using the API output, print each extracted entity and its type.


## Caching Strategies

When you are making a lot of requests to an API, it is a good idea to cache the results. We don't want to make the same request over and over again if we don't have to as this can effect our rate limits and thus our pricing.

Caching can be done in a number of ways. The simplest method is a Python dictionary where the key is the request and the value is the response. You can use Python `pickle` library to save the dictionary to disk so that you can use it across multiple sessions.

The caching strategy looks like this:

    1. Check if the request is in the cache
    2. If it is, we have a CACHE hit:
    3.    return the response from the ca
    4. else (cache miss)
    5. Call the API to get the response

Let's do an example with the Google Geocoding API on the IoT Portal. 


In [16]:
import requests
import requests_cache as rq

apikey = 'YOURAPIKEYHERE'
pickle_file = 'geocode_cache.pkl'
location = 'Syracuse, NY'
cache_key = location.lower()
cache = rq.clear_cache(pickle_file)
for i in range(3):
    if cache_key in cache.keys():
        print('Using cache')
        geo = cache[cache_key]

    else:
        print('Making request')
        url = 'https://cent.ischool-iot.net/api/google/geocode'
        headers = { 'X-API-KEY': apikey }
        params = {'location': location}
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        geo = response.json()
        cache[cache_key] = geo
        rq.save_cache(cache, pickle_file)

    print(geo['results'][0]['geometry']['location'])



Making request
{'lat': 43.0481221, 'lng': -76.14742439999999}
Using cache
{'lat': 43.0481221, 'lng': -76.14742439999999}
Using cache
{'lat': 43.0481221, 'lng': -76.14742439999999}


## Challenge 4-2-2

Come up with a caching strategy for the Azure sentiment API. Use the `requests_cache.py` module. You will need to decided on the cache key.

For the following input:

for each text in the list:  
output the text, the sentiment, and if the results came from the API or cache.

```
texts = [
    "I love IST356. It is the best course I've ever taken.", 
    "I hate the New York Giants.",
    "I love IST356. It is the best course I've ever taken.", 
    "I don't like the New York Giants."
]
```